In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from tensorflow.examples.tutorials.mnist import input_data
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import csv
import random

In [2]:
input_filename = 'text_file.txt'


def get_words_from_file(input_filename):
    f = open(input_filename,"r")
    buffer = f.read()
    words = buffer.split(' ')
    word_dict = {}
    word_list = []
    i = 0
    for word in words:
        if(not word_dict.__contains__(word)):
            word_dict[word] = i
            word_list.append(word)
            i += 1


    return word_dict, word_list

In [3]:
def get_n_words_random(word_list, n):
    random_index = random.randrange(1, len(word_list) -  n - 1)
    return np.array(range(random_index - 1,random_index + n - 1)), np.array(range(random_index + n - 1,random_index + n))

def get_n_words(word_list, start_index, n):
    random_index = start_index
    return np.array(range(random_index - 1,random_index + n - 1)), np.array(range(random_index + n - 1,random_index + n))

def get_training_data(word_list):
    train_x = []
    train_y = []
    for i in range(len(word_list)/batch_size):
        x, y = get_n_words(word_list, i*batch_size, n_steps)
        train_x.append(x)
        train_y.append(y)
    return np.array(train_x), np.array(train_y)


words_dict, words_list = get_words_from_file(input_filename)

In [4]:
n_inputs = 1
n_steps = 5
n_neurons = 15
n_outputs = 1
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])
#cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons, activation = tf.tanh)
#cell = tf.contrib.rnn.BasicLSTMCell(n_neurons, forget_bias=1.0)
#output_cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.BasicRNNCell(num_units = n_neurons, activation = tf.tanh), output_size = n_outputs)
#outputs , states = tf.nn.dynamic_rnn(output_cell, X, dtype= tf.float32)
#logits = tf.layers.dense(states, n_outputs)
#xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels= y , logits= logits)
with tf.variable_scope("lstm"):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_neurons,
                                             forget_bias=1.0)
    outputs, states = tf.nn.dynamic_rnn(lstm_cell,X, dtype=tf.float32)

weights = {
        'linear_layer': tf.Variable(tf.truncated_normal([n_neurons,
                                                         n_outputs],
                                                         mean=0,stddev=.01))
}


# Extract the last relevant output and use in a linear layer
final_output = tf.matmul(states[1],
                         weights["linear_layer"])
loss = tf.reduce_mean(tf.square(final_output - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
#optimizer = tf.train.RMSPropOptimizer(0.1, 0.9, 0.01)
training_op = optimizer.minimize(loss)
#correct = tf.nn.in_top_k(logits, y, 1)
#accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [5]:
def test(sess, outputs):
    random_index = 150
    num_of_words_to_predict = 50
    x_new = np.array(range(random_index,random_index + num_of_words_to_predict))
    batch_x = x_new.reshape((-1,n_steps,n_inputs))
    y_pred = sess.run(outputs, feed_dict={X: batch_x})
    final_output = tf.matmul(states[1],
                         weights["linear_layer"])
    print(final_output.eval(feed_dict={X: batch_x}))
    

In [6]:
n_iterations = 200
batch_size = 50
batches = 50
learning_rate = 0.01


init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_iterations):
        print "Epoch:", epoch
        for j in range(batches):  #((len(words_list) - n_steps)/batch_size):
            batch_x , batch_y = get_training_data(words_list)
            #print(batch_y.shape, batch_x.shape)
            batch_y =  batch_y.reshape((-1,n_outputs))
            batch_x = batch_x.reshape((-1,n_steps,n_inputs))
            #print(batch_y.shape, batch_x.shape)
            sess.run(training_op, feed_dict={X: batch_x, y: batch_y})
        if epoch % 10 == 0:
            mse = loss.eval(feed_dict={X: batch_x,y: batch_y})
            print ("Iter " + str(epoch) + ", Minibatch Loss= " + \
                    "{}".format(mse) )
            test(sess, outputs)
            

Epoch: 0
Iter 0, Minibatch Loss= 171811.1875
[[ 0.2534366 ]
 [ 0.25313166]
 [ 0.25282633]
 [ 0.2525208 ]
 [ 0.25221536]
 [ 0.25191042]
 [ 0.25160629]
 [ 0.25130326]
 [ 0.25100183]
 [ 0.25070229]]
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Iter 10, Minibatch Loss= 164724.578125
[[ 10.39970779]
 [ 10.399827  ]
 [ 10.39993286]
 [ 10.40002632]
 [ 10.40010738]
 [ 10.40017986]
 [ 10.40024281]
 [ 10.40029716]
 [ 10.40034676]
 [ 10.40038967]]
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Iter 20, Minibatch Loss= 159461.59375
[[ 18.13016891]
 [ 18.13023949]
 [ 18.13030052]
 [ 18.13035393]
 [ 18.13040161]
 [ 18.13044357]
 [ 18.13047981]
 [ 18.13051224]
 [ 18.13054085]
 [ 18.13056564]]
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Epoch: 30


Exception KeyboardInterrupt in <function _remove at 0x10e12d500> ignored


KeyboardInterrupt: 